### 📚 Bi-LSTM 구현하기

#### 💡 라이브러리 불러오기

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

--------------------------------------------------------------------------------------------------------

#### 💡 MNIST 데이터 불러오기

- 숫자 이미지 판별을 양방향 LSTM을 통해 예측
- 이미지는 주로 **(배치사이즈, 채널수, 이미지 너비, 이미지 높이)** 형태의 크기를 지니고 있다.
    - MNIST 데이터의 채널 수는 1이고, 이미지 크기가 28x28이므로 각 배치 데이터의 크기는 **(배치사이즈, 1, 28, 28)**
    - 이때 크기를 (배치사이즈, 28, 28)로 생각할 수 있다.
    - 이미지 픽셀의 각 열을 하나의 벡터로 보고, 행을 타임 스텝으로 본다면 **(배치 사이즈, 시계열의 길이, 벡터의 크기)**를 가진 시계열 데이터로 생각할 수 있다.
- 즉, 순환 신경망(RNN)도 이미지 처리에 활용될 수 있다.

In [2]:
tensor_mode = torchvision.transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root="./data", train=True, transform = tensor_mode, download=True)
testset = torchvision.datasets.MNIST(root="./data", train=False, transform = tensor_mode, download=True)

In [3]:
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

---------------------------------------------------------------------------------------------

#### 💡 Bi-LSTM 모델 구축하기

In [6]:
class BiLSTM(nn.Module) :
    def __init__(self, input_size, hidden_size, num_layers, seq_length, num_classes, device) :
        super(BiLSTM, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True) 
        self.fc = nn.Linear(seq_length*hidden_size*2, num_classes)
        
    def forward(self, x) :
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(-1, self.seq_length*self.hidden_size*2)
        out = self.fc(out)
        return out

- `init` 함수에서
    - 입력값의 크기(이미지의 열 크기), 은닉층의 노드수, 은닉층의 개수, 시계열의 길이(이미지의 행 크기), 클래스 수, gpu 활용 여부
- `lstm`
    - bidirectional = True : 양방향 LSTM을 생성
    - batch_first = True : 크기가 (배치 사이즈, 시계열의 길이, 입력값의 크기)를 지닌 데이터
- `fc` 
    - 모든 타임 스텝에 대한 LSTM 결과를 분류에 사용한다. 
    - 입력값의 크기는 **시계열의 길이 * 은닉층의 크기 * 2**
    - 양방향 LSTM은 정방향, 역방향에 대한 LSTM을 계산한 후 합칠 결과 (concatenate)를 사용
    - 따라서 각각의 은닉층 결과 2개가 합쳐지므로 2를 곱한다.
- `h0, c0`
    - 은닉 상태와 셀 상태의 초깃값을 정의한다. 
    - 여기서도 양방향에 대한 초깃값을 지정해야 하므로 은닉층의 개수에 2를 곱한다. 
- `out`
    - 모델에서 나온 out의 크기는 **(배치사이즈, 시계열의 길이, 은닉층의 노드수 * 2)**
    - 모든 데이터를 nn.Linear에 사용하기 위해 reshape하여 **(배치사이즈, 시계열의 길이 * 은닉층의 노드수 * 2)**로 변경
    - fc를 태워서 크기가 10(클래스 수)인 출력 벡터를 산출

---------------------------------------------------------------------------------------------

#### 💡 하이퍼 파라미터 정의하기

- 모델에 필요한 변수 정의
- 이미지 데이터의 행 : 시계열
- 이미지 데이터의 열 : 입력 벡터

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 이미지 데이터의 행 : 시계열
sequence_length = trainset.data.size(1)
# 이미지 데이터의 열 : 입력 벡터
input_size = trainset.data.size(2)

num_layers = 2
hidden_size = 12
num_classes = 10

---------------------------------------------------------------------------------------------

#### 💡 모델, 손실함수, 최적화 기법 정의하기

In [13]:
model = BiLSTM(input_size, hidden_size, num_layers, sequence_length, num_classes, device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 5e-3)

---------------------------------------------------------------------------------------------

#### 💡 모델 학습하기

- 원래 배치 데이터의 크기가 **(배치사이즈, 1, 28, 28)**이므로 `squeeze(1)`을 통해서 데이터의 크기를 **(배치사이즈, 28, 28)**로 변환
- 학습 도중 정확도를 구할 때에는 변수 업데이트가 필요 없으므로 `detach()`를 사용하여 outputs의 `requires_grad`를 비활성화

In [15]:
for epoch in range(51) :
    correct = 0
    total = 0
    for data in trainloader :
        optimizer.zero_grad()
        inputs, labels = data[0].to(device).squeeze(1), data[1].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.detach(), 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
    print("[%d] train acc: %.2f" %(epoch, 100*correct/total))

[0] train acc: 91.92
[1] train acc: 97.72
[2] train acc: 98.28
[3] train acc: 98.59
[4] train acc: 98.83
[5] train acc: 98.91
[6] train acc: 99.06
[7] train acc: 99.13
[8] train acc: 99.12
[9] train acc: 99.31
[10] train acc: 99.32
[11] train acc: 99.32
[12] train acc: 99.35
[13] train acc: 99.48
[14] train acc: 99.42
[15] train acc: 99.54
[16] train acc: 99.52
[17] train acc: 99.62
[18] train acc: 99.60
[19] train acc: 99.59
[20] train acc: 99.62
[21] train acc: 99.61
[22] train acc: 99.64
[23] train acc: 99.62
[24] train acc: 99.65
[25] train acc: 99.65
[26] train acc: 99.68
[27] train acc: 99.79
[28] train acc: 99.72
[29] train acc: 99.65
[30] train acc: 99.66
[31] train acc: 99.73
[32] train acc: 99.83
[33] train acc: 99.74
[34] train acc: 99.66
[35] train acc: 99.78
[36] train acc: 99.78
[37] train acc: 99.75
[38] train acc: 99.74
[39] train acc: 99.89
[40] train acc: 99.73
[41] train acc: 99.65
[42] train acc: 99.81
[43] train acc: 99.80
[44] train acc: 99.82
[45] train acc: 99.8

---------------------------------------------------------------------------------------------

#### 💡 모델 평가하기

In [30]:
def accuracy(dataloader) :
    correct = 0
    total = 0
    with torch.no_grad() :
        model.eval()
        for data in dataloader :
            inputs, labels = data[0].to(device).squeeze(1), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
    acc = 100*correct/total
    model.train()
    return acc

In [31]:
train_acc = accuracy(trainloader)
test_acc = accuracy(testloader)
print("Train ACC : %.1f, Test ACC : %.1f" %(train_acc, test_acc))

Train ACC : 100.0, Test ACC : 98.6
